<a href="https://colab.research.google.com/github/smduarte/spbd-2425/blob/main/docs/labs/lab1/SPBD_Labs_mapreduce1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python MapReduce Example

Word count implemented in pure Python.

This notebook exemplifies the execution of a map-reduce program in Python, using Hadoop.
In this example, hadoop runs in standalone mode and reads data from the local filesystem, while in cluster mode data is read typically from HDFS dsitributed file system.


### Download the dataset

In [ ]:
!wget -q -O os_maias.txt https://www.dropbox.com/s/n24v0z7y79np319/os_maias.txt?dl=0

## WordCount Example
Read the words from input and count them.

The processing is split into two steps:

+ The mapper emits for each line the number of words
+ The reduces sums all the tuples produced by the mapper stage...

### Mapper

By starting an element with "%%file", you are specifying that when run, the contents are written to the local disk.

In [ ]:
%%file mapper.py
#!/usr/bin/env python

# import sys
import sys
# import string library function
import string

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # remove punctuation characters
    line = line.translate(str.maketrans('', '', string.punctuation+'«»'))
    # split the line into words
    words = line.split()
    print('words\t%s' % len(words))

### Reducer

In [ ]:
%%file reducer.py
#!/usr/bin/env python

import sys

total_count = 0

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    key, count = line.split('\t', 1)

    # convert count (currently a string) to int
    count = int(count)

    total_count += count

print('words\t%s' % (total_count))

## Local execution

The scripts can be tested using just the unix shell, as follows...

### Make the scripts executable

In [ ]:
!chmod a+x mapper.py && chmod a+x reducer.py

### Execute

The execution workflow is as follows:

+ The input file is piped into the input of the mapper;
+ The output the mapper is sorted;
+ The sorted output of the mapper is fed to the reducer stage.

In [ ]:
!cat "os_maias.txt" | ./mapper.py | sort -k1,1 | ./reducer.py

## MapReduce with HADOOP

In [ ]:
#@title Install Hadoop on Google Colab
!curl -s https://raw.githubusercontent.com/smduarte/spbd-2425/main/scripts/install_hadoop.sh | bash

## Hadoop standalone mode execution

For executing in an hadoop cluster, input data should be moved into an HDFS directory. For executing in standalone mode, data can be read from the local filesystem.


The output directory needs to be cleared...

In [ ]:
rm -rf results

### Submitting the job

The _hadoop_ command is used to submit the mapreduce job to the cluster...

In [ ]:
!hadoop jar /usr/local/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py -input os_maias.txt -output results

#### Checking the results
The result is stored in directory results.

In [ ]:
!cat results/part-*

# MrJob MapReduce Python Example

Word count implemented in pure Python, using the library MrJob.

[MrJob](https://mrjob.readthedocs.io/en/latest/) can be used to write MapReduce jobs and run them on several platforms.

Some key advantages:
+ Write **multi-step** MapReduce jobs in pure Python;
+ Test on your **local machine**;
+ Deploy jobs in several cloud plataforms of several vendors.

In [4]:
#@title Install MrJob
!pip install mrjob --quiet
!wget -q -O /etc/mrjob.conf https://raw.githubusercontent.com/smduarte/spbd-2425/main/scripts/mrjob.conf

### MrJob WordCount Example
Read the words from input and count them.

The processing is split into two main phases:

+ The mapper emits for each line the number of words
+ The reduces sums all the tuples produced by the mapper stage...

Using MrJob, a MapReduce job can be expressed in a single Python class,
with methods for each of the phases. The reducer phase is called separately for each key, with the collection of values to be reduced.

In [5]:
%%file wordcount.py

import string
from mrjob.job import MRJob

class MRWordCount(MRJob):

    def mapper(self, _, line):
      # remove leading and trailing whitespace
      line = line.strip()
      # remove punctuation characters
      line = line.translate(str.maketrans('', '', string.punctuation+'«»'))
      # split the line into words
      yield "words", len(line.split())

    def reducer(self, key, values):
        yield key, sum(values)

if __name__ == '__main__':
    MRWordCount.run()

Overwriting wordcount.py


## Execution of MrJob programs


In [8]:
#@title Local Execution (Option 1)
import importlib
import wordcount
importlib.reload(wordcount)

# prepare the mapreduce job for local execution
mr_job = wordcount.MRWordCount(args=['-r', 'local','os_maias.txt'])

# execute the job and print the output results
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print( key, value)

words 213359


In [10]:
#@title Local Execution (Option 2)

!python  wordcount.py -r local os_maias.txt

Using configs in /etc/mrjob.conf
Creating temp directory /tmp/wordcount.root.20240917.151305.253333
Running step 1 of 1...
job output is in /tmp/wordcount.root.20240917.151305.253333/output
Streaming final output from /tmp/wordcount.root.20240917.151305.253333/output...
"words"	213359
Removing temp directory /tmp/wordcount.root.20240917.151305.253333...


## Supplying a combiner...

In [33]:
%%file wordcount_with_combiner.py

import string
from mrjob.job import MRJob

class MRWordCountWithCombiner(MRJob):

    def mapper(self, _, line):
      # remove leading and trailing whitespace
      line = line.strip()
      # remove punctuation characters
      line = line.translate(str.maketrans('', '', string.punctuation+'«»'))
      # split the line into words
      yield "words", len(line.split())

    def combiner(self, key, values):
        yield key, sum(values)

    def reducer(self, key, values):
        yield key, sum(values)

if __name__ == '__main__':
    MRWordCountWithCombiner.run()

Writing wordcount_with_combiner.py


In [34]:
#@title Local Execution

import wordcount_with_combiner

import importlib
importlib.reload(wordcount_with_combiner)

# prepare the mapreduce job for local execution
mr_job = wordcount_with_combiner.MRWordCountWithCombiner(args=['-r', 'local','os_maias.txt'])

# execute the job and print the output results
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print( key, value)

words 213359
